In [6]:
import pandas as pd
import os
import time

from dataclasses import dataclass
from subgrounds.subgraph.subgraph import Subgraph
from subgrounds.schema import TypeRef
from subgrounds.subgrounds import Subgrounds
from subgrounds.subgraph.fieldpath import FieldPath
from subgrounds.subgraph import SyntheticField


pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [7]:
@dataclass
class SubgroundsFirehose:
    '''
    Wrapper/Helper class built ontop of Subgrounds that allows user to easily access the subgraph schema and query the subgraph for all possible queries.

    # There are three components for the Subgrounds pipeline:
    # 1) Load Subgrounds - The main entry point for the pipeline, Subgrounds is used to query the subgraph and load the data into dataframes.
    # 2) Load Subgraph -  Load the schema data from a graphql endpoint. Specifically the schema data pertaining to `Query`. 
    # 3) Define Query Path - The query path is defined automatically to run every possible query on the subgraph and returns a list of dataframes.
    '''
    sub: Subgrounds
    endpoint: str
    subgraph: Subgraph
    # TODO - turn this into a dictionary with respective query field name for easier lookup. THis would be a nice to have, but the names are also saved in the directory too already so not sure this is necessary.
    data_list = []

    def runFirehose(self):
        '''
        Use runFirehose() get all queryable fields from a subgraph. Firehose will run every possible query on the subgraph and return a list of dataframes.
        '''
    # 1) get schema list 
        schema_list = self.getSubgraphSchema(self.subgraph)
    # 2) get schema query field list
        query_field_list = self.getSchemaFields(self.subgraph, schema_list[schema_list.index('Query')])

    # 3) get field paths from 2
    # 4) filter, format, and sort. End result is subgraph_query_field_list -> sorted_subgraph_field_query_list steps
        filtered_schema_list = [x for x in schema_list if not x.endswith('_') and x != 'Query' and x != 'Subscription']
        # format query fields
        fmt_query_field_list = [self.formatFieldStr(field) for field in query_field_list]
        # drop empty values if there are any. Empty fields will break the query.
        fmt_query_field_list = [field for field in fmt_query_field_list if field != '']
        # make a query field list with queryable fields - these are plural values that end with 's'.
        subgraph_query_field_list = [field for field in fmt_query_field_list if field.endswith('s')]
        # make a subgraph schema list with schema objects that relate to the query fields. These are singular values that do not end with 's'.
        subgraph_schema_query_list = [field[0].upper() + field[1:] for field in filtered_schema_list if not field.endswith('s')]

    # 4b) order the field and schema lists
        # order subgraph_query_field_list by alphabet values.
        sorted_subgraph_query_field_list = sorted(subgraph_query_field_list, key=lambda x: x.rstrip('s'))
        # order subgraph schema list by alphabet values. If '_' is first character, sort by the next strig index.
        sorted_subgraph_schema_query_list = sorted(subgraph_schema_query_list, key=lambda x: x.lstrip('_'))

        # PRINT HELP DEBUG STATEMENTS. CURRENTLY DISABLED
        # print(f'subgraph_schema_query_list and subgraph_query_field_list (ordered)')
        # for i in range(len(subgraph_query_field_list)):
        #     print(f'{sorted_subgraph_schema_query_list[i]}, {sorted_subgraph_query_field_list[i]}')
    
        # get values of endpoint after the last /
        subgraph_name = self.endpoint.split('/')[-1]

        outer_start = time.time()
        # automate querying subgraph data for each "queryable" subgraph schema object
        for i in range(len(sorted_subgraph_query_field_list)):
            # start query timer
            start = time.time()
            # get query field path
            field_list = self.getSchemaFields(self.subgraph, sorted_subgraph_schema_query_list[i])
            field_path = self.getFieldPath(self.subgraph, sorted_subgraph_query_field_list[i])
            # end query timer
            field_path_params = field_path(first=5)
            
            df = self.sub.query_df(field_path_params, field_list)
            
            # make a directory name of subgraph name and query field list. 
            # This will be used to create a directory to store the dataframes
            dir_name = f'{subgraph_name}'

            # make a directory to store the dataframes if directory doesn't exist
            if not os.path.exists(dir_name):
                os.mkdir(dir_name)

            # save the dataframe to the directory
            df.to_csv(f'{dir_name}/{sorted_subgraph_query_field_list[i]}.csv')

            self.data_list.append(df)
            end = time.time()
            # PRINT DEBUG TO UNDERSTAND WHAT IS BEING QUERIED
            # print(f'Round {i} - query time was {end - start: .3f} seconds \nschema {sorted_subgraph_schema_query_list[i]} with {sorted_subgraph_query_field_list[i]}')
            # print(f'There are {len(field_list)} fields in the query - {field_list}. \nLength of df is {len(df)}.')
        outer_end = time.time()
        print(f'{subgraph_name} query took {outer_end - outer_start: .3f} seconds. Running total: {len(self.data_list)} schema dataframes and {sum([len(df) for df in self.data_list])} total rows retrieved.')


    def getFieldPath(self, subgraph: Subgraph, field: str,  operation: str ='Query') -> FieldPath:
        '''
        Use getFieldPath to get a FieldPath from a string.

        Args:
            subgraph (Subgraph) = Subgraph object with a loaded graphql endpoint
            field (str) = Enter the string that will be converted to a FieldPath
            operation (str) = Enter one of the following - 'Query', 'Mutation', or 'Subscription'. Default is 'Query' because that is most commonly used.
        Returns:
            FieldPath = FieldPath object
        '''
        return subgraph.__getattribute__(operation).__getattribute__(field)

    def getSubgraphSchema(self, subgraph: Subgraph) -> list[str]:
        '''
        Use getSubgraphSchema to fetch a schema list from a subgraph object.

        Args:
            subgraph (Subgraph) = Subgraph object with a loaded graphql endpoint
        Returns:
            list[str] = schema objects list from subgraph
        '''
        return list(name for name, type_ in subgraph._schema.type_map.items() if type_.is_object)

    def getSchemaFields(self, subgraph: Subgraph, schema_object: str) -> list[str]:
        '''
        Use getSubgraphField to get a list of SchemaFields from a subgraph.

        Args:
            subgraph (Subgraph) = Subgraph object with a loaded graphql endpoint
            schema_object (str) = Enter the schama object string that you want to get all fields for
        Returns:
            list[str] = list of field strings from schema_object
        '''
        return list(field.name for field in subgraph.__getattribute__(schema_object)._object.fields)

    def formatFieldStr(self, field: str) -> str:
        # if value does not end with a s and does not start and end with _, make the first non _ character in the string a capital letter
        '''
        Use formatFieldStr to format the field string to be queryable. In order to make the field string queryable, the first non _ character must be capitalized.

        Args:
            field (str) = field string
        Returns:
            str = formatted field string
        '''
        if not field.endswith('s') and not field.startswith('_'):
            field = field[0].upper() + field[1:]
        return field


In [8]:
hosted_query_ids = [
                    'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
                    'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-optimism',
                    'https://api.thegraph.com/subgraphs/name/messari/abracadabra-money-arbitrum',
                    'https://api.thegraph.com/subgraphs/name/messari/abracadabra-money-avalanche',
                    'https://api.thegraph.com/subgraphs/name/messari/abracadabra-money-bsc',
                    'https://api.thegraph.com/subgraphs/name/messari/abracadabra-money-fantom',
                    'https://api.thegraph.com/subgraphs/name/messari/abracadabra-money-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/aave-arc-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/aave-amm-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/aave-rwa-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/aave-v2-avalanche',
                    'https://api.thegraph.com/subgraphs/name/messari/aave-v2-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/arrakis-finance-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/curve-finance-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/balancer-v2-arbitrum',
                    'https://api.thegraph.com/subgraphs/name/messari/balancer-v2-polygon',
                    'https://api.thegraph.com/subgraphs/name/messari/balancer-v2-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-arbitrum',
                    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-avalanche',
                    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-bsc',
                    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-celo',
                    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-fantom',
                    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-fuse',
                    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-harmony',
                    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-polygon',
                    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-moonbeam',
                    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-moonriver',
                    'https://api.thegraph.com/subgraphs/name/messari/sushiswap-gnosis',
                    'https://api.thegraph.com/subgraphs/name/messari/aave-governance',
                    'https://api.thegraph.com/subgraphs/name/messari/compound-governance-v1',
                    'https://api.thegraph.com/subgraphs/name/messari/code4rena-governance',
                    'https://api.thegraph.com/subgraphs/name/messari/indexed-governance',
                    'https://api.thegraph.com/subgraphs/name/messari/euler-governance',
                    'https://api.thegraph.com/subgraphs/name/messari/inverse-finance-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/liquity-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/qidao-fantom',
                    'https://api.thegraph.com/subgraphs/name/messari/qidao-polygon',
                    'https://api.thegraph.com/subgraphs/name/messari/qidao-moonriver',
                    'https://api.thegraph.com/subgraphs/name/messari/qidao-optimism',
                    'https://api.thegraph.com/subgraphs/name/messari/qidao-gnosis',
                    'https://api.thegraph.com/subgraphs/name/messari/opensea-v1-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/opensea-v2-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/opensea-seaport-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/tokemak-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/beefy-finance-arbitrum',
                    'https://api.thegraph.com/subgraphs/name/messari/beefy-finance-avalanche',
                    'https://api.thegraph.com/subgraphs/name/messari/beefy-finance-aurora',
                    'https://api.thegraph.com/subgraphs/name/messari/beefy-finance-celo',
                    'https://api.thegraph.com/subgraphs/name/messari/gamma-ethereum',
                    'https://api.thegraph.com/subgraphs/name/messari/gamma-polygon',
                    # non-messari subgraphs
                    'https://api.thegraph.com/subgraphs/name/nissoh/gmx-arbitrum',
                    'https://api.thegraph.com/subgraphs/name/premiafinance/premia',
                    'https://api.thegraph.com/subgraphs/name/synthetixio-team/synthetix'
                    ]

In [9]:
sub_firehose_data = []
counter = 0
start = time.time()
for endpoint in hosted_query_ids:
    counter += 1
    print(f'query {counter} for {endpoint}')
    sub = Subgrounds()
    subgraph = sub.load_subgraph(endpoint)
    subFirehose = SubgroundsFirehose(sub, endpoint, subgraph)
    subFirehose.runFirehose()
    sub_firehose_data.append(subFirehose)
end = time.time()

print(f'total query time took {end - start: .3f} seconds')

query 1 for https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum
uniswap-v3-arbitrum query took  19.007 seconds. Running total: 20 schema dataframes and 87 total rows retrieved.
query 2 for https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-optimism
uniswap-v3-optimism query took  27.668 seconds. Running total: 40 schema dataframes and 174 total rows retrieved.
query 3 for https://api.thegraph.com/subgraphs/name/messari/abracadabra-money-arbitrum
abracadabra-money-arbitrum query took  11.323 seconds. Running total: 62 schema dataframes and 264 total rows retrieved.
query 4 for https://api.thegraph.com/subgraphs/name/messari/abracadabra-money-avalanche
abracadabra-money-avalanche query took  13.685 seconds. Running total: 84 schema dataframes and 361 total rows retrieved.
query 5 for https://api.thegraph.com/subgraphs/name/messari/abracadabra-money-bsc
abracadabra-money-bsc query took  10.779 seconds. Running total: 106 schema dataframes and 454 total rows retriev

In [23]:
# get folder contents from synthetix folder in os
synthetix_folder = os.listdir('synthetix')
print(synthetix_folder)

['synthHolders.csv', 'feesClaimeds.csv', 'dailyBurneds.csv', 'issuers.csv', 'rewardEscrowHolders.csv', 'dailyIssueds.csv', 'synthetixes.csv', 'burneds.csv', 'totalActiveStakers.csv', 'totalDailyActiveStakers.csv', 'activeStakers.csv', 'issueds.csv', 'snxholders.csv', 'debtSnapshots.csv']


In [25]:
uniswap_v3_arbitrum_folder = os.listdir('uniswap-v3-arbitrum')
print(uniswap_v3_arbitrum_folder)

['activeAccounts.csv', 'liquidityPoolDailySnapshots.csv', 'usageMetricsHourlySnapshots.csv', 'usageMetricsDailySnapshots.csv', 'liquidityPoolHourlySnapshots.csv', 'accounts.csv', 'dexAmmProtocols.csv', 'liquidityPoolFees.csv', 'protocols.csv', 'deposits.csv', 'tokens.csv', 'withdraws.csv', 'liquidityPools.csv', 'financialsDailySnapshots.csv', 'tokenWhitelists.csv', 'liquidityPoolAmounts.csv', 'rewardTokens.csv', 'swaps.csv', 'events.csv', 'helperStores.csv']


In [26]:
# get synthHolders.csv from pandas dataframe in synthetix folder
synthHolders = pd.read_csv('synthetix/synthHolders.csv')

In [27]:
# get activeAccounts.csv from pandas dataframe in uniswap-v3-arbitrum folder
accountHolders = pd.read_csv('uniswap-v3-arbitrum/activeAccounts.csv')

In [28]:
synthHolders

,Unnamed: 0,id,synth,balanceOf
0,0,0x0000000000000000000000000000000000000001-sUSD,sUSD,100.106245
1,1,0x000000000000000000000000000000000000dead-sUSD,sUSD,4.051033
2,2,0x000000000000006f6502b7f2bbac8c30a3f67e9a-sUSD,sUSD,0.000000
3,3,0x0000000000000eb4ec62758aae93400b3e5f7f18-sUSD,sUSD,0.000000
4,4,0x00000000000017c75025d397b91d284bbe8fc7f2-sUSD,sUSD,0.000000


In [29]:
accountHolders

,Unnamed: 0,id
0,0,0x000000000000d34c44564053af35e4fe271d0caa-19240
1,1,0x000000000000d34c44564053af35e4fe271d0caa-19241
2,2,0x000000000000d34c44564053af35e4fe271d0caa-19242
3,3,0x000000000000d34c44564053af35e4fe271d0caa-19243
4,4,0x000000000000d34c44564053af35e4fe271d0caa-19244
